In [ ]:
import ee
import geemap

In [ ]:
ee.Initialize()

In [ ]:
Map = geemap.Map()
# roi = geemap.shp_to_ee(r"中国.shp") \
#                 .filter(ee.Filter.eq('name', '江西省')) \
#                 .geometry() \
#                 .buffer(1000)
roi = geemap.shp_to_ee(r"市.shp").geometry().buffer(1000)

In [ ]:
startdata = '2022-01-01'
enddata = '2022-12-31'

### data prepare

In [ ]:
Climate_collection = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filter(ee.Filter.date(startdata, enddata)).select(['pr','tmmn','tmmx','srad'])   # 分辨率 4638.3 meters
Landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.date(startdata, enddata))
Landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filter(ee.Filter.date(startdata, enddata))
# ndvi_col = ee.ImageCollection("projects/ee-qingqiu760/assets/2020ndvi")
# ndvi_col = ee.ImageCollection("projects/first-renderer-465613-q7/assets/2024ndvi")
# imgCLCD = ee.Image('projects/lulc-datase/assets/LULC_HuangXin/CLCD_v01_2023')
# Landcover_collection = ee.ImageCollection('MODIS/061/MCD12Q1').select('LC_Type1')   # 分辨率 500 m
# Landcover = ee.Image('projects/ee-qingqiu760/assets/plant2020')
Landcover = ee.Image('projects/ee-qingqiu760/assets/plant2024')

### 数据集的预处理与重新投影

In [ ]:
# def reprojectclip(image):
#     return image.reproject(crs='EPSG:4326',scale=30).clip(roi)

# 处理气候变量
def solar_radiation_qc(image):
    srad = image.select('srad').multiply(0.1).multiply(2.592)  # 单位转化
    tmmn = image.select('tmmn').multiply(0.1)
    tmmx = image.select('tmmx').multiply(0.1)
    pr = image.select('pr')
    tmean = tmmn.add(tmmx).divide(2).rename('tmean')
    # 提取时间信息
    datestr = image.get('system:time_start')
    year = ee.Date(datestr).get('year')
    month = ee.Date(datestr).get('month')
    # 计算I_month，I_month = (tmean / 5) ^ 1.514
    I_month = tmean.divide(5).pow(1.514).rename('I_month')

    return image.addBands(srad,None,True).addBands(tmmn,None,True).addBands(tmmx,None,True).addBands(pr,None,True).addBands(tmean).addBands(I_month)\
        .reproject(crs='EPSG:4326',scale=30).clip(roi).set('year',year).set('month',month).set('yearmonth',ee.Date(datestr).format('YYYYMM'))

Climate_collection = Climate_collection.map(solar_radiation_qc)#.map(reprojectclip)
# Landcover = imgCLCD.reproject(crs='EPSG:4326',scale=30).clip(roi)
# Landcover_collection = Landcover_collection#.map(reprojectclip)

In [ ]:
# Climate_collection.first().bandNames().getInfo()

### QA去云

In [ ]:
def masksr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    #thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True)\
        .updateMask(qaMask)\
        .updateMask(saturationMask)

# modis 去云函数
def mask_clouds(image):
    # 选择QA波段
    QA = image.select('state_1km')
    
    # 创建掩膜位掩盖不同的干扰
    internal_cloud_flag = 1 << 10  # 内部云算法标志
    cloud_shadow_flag = 1 << 2  # 云阴影标志
    snow_ice_flag = 1 << 11  # 雪/冰覆盖标志
    
    # 返回同时屏蔽云、云阴影和雪/冰区域的图像
    mask = QA.bitwiseAnd(internal_cloud_flag).eq(0) \
           .And(QA.bitwiseAnd(cloud_shadow_flag).eq(0)) \
           .And(QA.bitwiseAnd(snow_ice_flag).eq(0))
    
    return image.updateMask(mask)

def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000) \
    .select("B.*") \
    .copyProperties(image, ["system:time_start"])



In [ ]:
merge_image = ee.ImageCollection(Landsat8.merge(Landsat9)).map(masksr).select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6']).filterBounds(roi)

In [ ]:
# 哨兵数据填补
year = 2022
s_lst = []
for month in range(1, 13):
    time = merge_image.filterDate(ee.Date(f'{year}-{month}-01'), ee.Date(f'{year}-{month}-01').advance(1, 'month')).first().get('system:time_start')
    landsat = merge_image.filterDate(ee.Date(f'{year}-{month}-01'), ee.Date(f'{year}-{month}-01').advance(1, 'month'))

    landsat_before = ee.ImageCollection(
            ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(ee.Date(f'{year-1}-{month}-01'), ee.Date(f'{year-1}-{month}-01').advance(1, 'month'))\
            .merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(ee.Date(f'{year-1}-{month}-01'), ee.Date(f'{year-1}-{month}-01').advance(1, 'month')))
    ).filterBounds(roi).map(masksr).select(['SR_B2', 'SR_B3','SR_B4', 'SR_B5', 'SR_B6'])

    landsat_after  = ee.ImageCollection(
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(ee.Date(f'{year+1}-{month}-01'), ee.Date(f'{year+1}-{month}-01').advance(1, 'month'))\
        .merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(ee.Date(f'{year+1}-{month}-01'), ee.Date(f'{year+1}-{month}-01').advance(1, 'month')))
    ).filterBounds(roi).map(masksr).select(['SR_B2', 'SR_B3','SR_B4', 'SR_B5', 'SR_B6'])
    
    sentiel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterDate(ee.Date(f'{year}-{month}-01'), ee.Date(f'{year}-{month}-01').advance(1, 'month')).map(maskS2clouds) \
                .select(['B2', 'B3','B4', 'B8', 'B11'], ['SR_B2', 'SR_B3','SR_B4', 'SR_B5', 'SR_B6']) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
                .filterBounds(roi)
    # if sentiel.bandNames().getInfo():
    #     unmask = landsat.unmask(landsat_before).unmask(landsat_after).unmask(sentiel).set('system:time_start', time)
    # else:
    #     unmask = landsat.unmask(landsat_before).unmask(landsat_after).set('system:time_start', time)
    # s_lst.append(unmask)
    imgcol = landsat.merge(landsat_before).merge(landsat_after).merge(sentiel)
    s_lst.append(imgcol.median().set('system:time_start', time))

    
unmask_sentiel = ee.ImageCollection(s_lst)

In [ ]:
Map.addLayer(ee.Image(unmask_sentiel.toList(12).get(0)).clip(roi), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 'unmask_sentiel')
Map

In [ ]:
# a = merge_image.filterDate('2023-09-01', '2023-09-30').max()
# # 可视化参数
# visualization = {
#   'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
#   'min': 0.0,
#   'max': 0.3,
# }
# Map.addLayer(a, visualization, 'a')
# Map

### 根据土地利用类型设置LUE、NDVI最小值、NDVI最大值、SR最小值或SR最大值

In [ ]:
# 相关设置参考论文：Simulation of maximum light use efficiency for some typical vegetation types in China
def generate_LUE(image):
    image = image.where(image.eq(3),0.485).where(image.eq(1),0.389).\
                where(image.eq(4),0.692).where(image.eq(2),0.985).\
                where(image.eq(5),0.768).where(image.eq(6),0.429).\
                where(image.eq(7),0.429).where(image.eq(8),0.542).\
                where(image.eq(9),0.542).where(image.eq(10),0.542).\
                where(image.eq(11),0.542).where(image.eq(12),0.542).\
                where(image.eq(13),0.542).where(image.eq(14),0.542).\
                where(image.eq(15),0.542).where(image.eq(16),0.542).where(image.eq(17),0.542)
    return image.toFloat()

def set_NDVImin(image):
    image = image.where(image.gt(0),0.023)
    return image.toFloat()

def set_NDVImax(image):
    image = image.where(image.eq(3),0.738).where(image.eq(1),0.647).\
                where(image.eq(2),0.676).where(image.eq(4),0.747).\
                where(image.eq(6),0.636).where(image.eq(7),0.636).\
                where(image.eq(8),0.636).where(image.eq(9),0.636).\
                where(image.gt(9),0.634).where(image.eq(5),0.702)
    return image.toFloat()

def set_SRmin(image):
  image = image.where(image.gt(0),1.05)
  return image.toFloat()

def set_SRmax(image):
  image = image.where(image.eq(3),6.63).where(image.eq(1),4.67).\
                where(image.eq(2),5.17).where(image.eq(4),6.91).\
                where(image.eq(6),4.49).where(image.eq(7),4.49).\
                where(image.eq(8),4.49).where(image.eq(9),4.49).\
                where(image.gt(9),4.44).where(image.eq(5),5.85)
  return image.toFloat()

In [ ]:
# LUE = Landcover_collection.map(generate_LUE).select(['LC_Type1'],['LUE'])
# NDVImin = Landcover_collection.map(set_NDVImin).select(['LC_Type1'],['NDVI_min'])
# NDVImax = Landcover_collection.map(set_NDVImax).select(['LC_Type1'],['NDVI_max'])
# SRmin = Landcover_collection.map(set_SRmin).select(['LC_Type1'],['SR_min'])
# SRmax = Landcover_collection.map(set_SRmax).select(['LC_Type1'],['SR_max'])
LUE = generate_LUE(Landcover).rename('LUE')
NDVImin = set_NDVImin(Landcover).rename('NDVI_min')
NDVImax = set_NDVImax(Landcover).rename('NDVI_max')
SRmin = set_SRmin(Landcover).rename('SR_min')
SRmax = set_SRmax(Landcover).rename('SR_max')

### 计算指数

In [ ]:
def Cal_index(image):
    Blue = image.select('SR_B2')
    Green = image.select('SR_B3')
    red = image.select('SR_B4')
    Nir = image.select('SR_B5')
    Swir = image.select('SR_B6')

    NDVI = Nir.subtract(red).divide(Nir.add(red))
    NDVI = NDVI.where(NDVI.gt(1),1).where(NDVI.lt(-1),-1)
    NDWI = Green.subtract(Nir).divide(Green.add(Nir))
    NDWI = NDWI.where(NDWI.gt(1),1).where(NDWI.lt(-1),-1)
    NDBI = Swir.subtract(Nir).divide(Swir.add(Nir))
    NDBI = NDVI.where(NDBI.gt(1),1).where(NDBI.lt(-1),-1)
    NIRV = NDVI.multiply(Nir)
    SR = ee.Image(1).add(NDVI).divide(ee.Image(1).subtract(NDVI))

    return image.addBands(NDVI.rename('NDVI')).addBands(NDWI.rename('NDWI')).addBands(NDBI.rename('NDBI')).addBands(NIRV.rename('NIRV')).addBands(SR.rename('SR'))

landsat_indices = unmask_sentiel.map(Cal_index).select(['NDVI','NDWI','NDBI','NIRV','SR'])
# landsat_indices = unmask_sentiel.map(Cal_index).select(['NDWI','NDBI','NIRV','SR'])

## 添加本地 NDVI

In [ ]:
def add_month(img):
    date = ee.Date(img.get('system:time_start'))
    return img.set({
        'year': date.get('year'),
        'month': date.get('month'),
        'month_id': date.format('YYYY-MM')
    })

In [ ]:
landsat_indices = landsat_indices.map(add_month)
# ndvi_col = ndvi_col.map(add_month)

### combine NDVImin max SRmin max

In [ ]:
def setyear(image):
    year_str = image.get('system:time_start')
    year = ee.Date(year_str).get('year')
    month = ee.Date(year_str).get('month')
    return image.set('year',year).set('month',month)

# LUE = LUE.map(setyear)
# NDVImin = NDVImin.map(setyear)
# NDVImax = NDVImax.map(setyear)
# SRmin = SRmin.map(setyear)
# SRmax = SRmax.map(setyear)
LUE = setyear(LUE)
NDVImin = setyear(NDVImin)
NDVImax = setyear(NDVImax)
SRmin = setyear(SRmin)
SRmax = setyear(SRmax)
landsat_indices = landsat_indices.map(setyear)

In [ ]:
filterTimeEqminmax = ee.Filter.equals(**{
    'leftField':'year',
    'rightField':'year'
})

innerJoin = ee.Join.inner()
# innerJoinedIL_LUE = innerJoin.apply(landsat_indices, LUE, filterTimeEqminmax)
# Indices_LUE = ee.ImageCollection(innerJoinedIL_LUE.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

# innerJoinedILL_Nm = innerJoin.apply(Indices_LUE, NDVImin, filterTimeEqminmax)
# Indices_LLM = ee.ImageCollection(innerJoinedILL_Nm.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

# innerJoinedILL_NN = innerJoin.apply(Indices_LLM, NDVImax, filterTimeEqminmax)
# Indices_LLM = ee.ImageCollection(innerJoinedILL_NN.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

# innerJoinedILL_NNS = innerJoin.apply(Indices_LLM, SRmin, filterTimeEqminmax)
# Indices_LLM = ee.ImageCollection(innerJoinedILL_NNS.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

# innerJoinedILL_NNSR = innerJoin.apply(Indices_LLM, SRmax, filterTimeEqminmax)
# Indices_LLM = ee.ImageCollection(innerJoinedILL_NNSR.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))
Indices_LUE = landsat_indices.map(lambda img: img.addBands(LUE))
Indices_LLM = Indices_LUE.map(lambda img: img.addBands(NDVImin)) \
                         .map(lambda img: img.addBands(NDVImax)) \
                         .map(lambda img: img.addBands(SRmin)) \
                         .map(lambda img: img.addBands(SRmax))


In [ ]:
# Map.addLayer(Indices_LLM, {}, 'Indices_LLM')
# Map

### 计算最优温度 (Topt)

In [ ]:
# Topt 为某一区域一年内 NDVI 值到达最高时的当月平均气温（单位℃），在此根据经验设定为每年的7月平均气温
start = ee.Date('2022-01-01')
end = ee.Date('2022-12-31')
n_months = end.difference(start, 'month') # .subtract(1)
months = ee.List.sequence(0, n_months).map(lambda n : start.advance(n, 'month'))

# def makeMonthlyComposite_NDVI(date):
#     date = ee.Date(date)
#     return NDVI_LST.filterDate(date, date.advance(1, 'month')).mean().set("system:index", date.format("YYYYMM")).set("year", date.format("YYYY")).set("month", date.format("MM"))
T = Climate_collection.filter(ee.Filter.eq('month', 7)).select(['tmean'],['T'])

def getTopt(image):
  tmean = image.select('T')

  Topt_mean = ee.Number(tmean.reduceRegion(**{
            'reducer': ee.Reducer.mean(),
            'geometry' :roi,          
            'scale': 30,
            'maxPixels': 1e9
            }).values().get(0))

  Topt = ee.Image(Topt_mean).rename('Topt')

  return image.addBands(Topt).clip(roi)

T = T.map(getTopt).select('Topt')


### 计算热量指标

In [ ]:
I = Climate_collection.select('I_month')
n_years = end.difference(start, 'year') # .subtract(1)
years = ee.List.sequence(0, n_years).map(lambda n : start.advance(n, 'year'))

def makeYearlyComposite_I(date):
  date = ee.Date(date)
  return I.filterDate(date, date.advance(1, 'year')).sum().set('system:index',date.format("YYYY")).set('year',date.get('year'))\
    .set('system:time_start',date.millis()).set('system:time_end',date.advance(1, 'year').millis())

I = ee.ImageCollection.fromImages(years.map(makeYearlyComposite_I)).select(['I_month'],['I'])
# Map.addLayer(I,{},'I')

### 月复合影像

In [ ]:
def makeMonthlyComposite(date):
    date = ee.Date(date)
    return Indices_LLM.filterDate(date, date.advance(1, 'month')).max().set("system:index", date.format("YYYYMM")).set("year", date.format("YYYY")).set("system:time_start", date.millis()).set("system:time_end", date.advance(1, 'month').millis())

Indices_LLM = ee.ImageCollection.fromImages(months.map(makeMonthlyComposite))
landsat_SR = Climate_collection.combine(Indices_LLM)

innerJoinedTopt = innerJoin.apply(landsat_SR, T, filterTimeEqminmax)
landsat_SR = ee.ImageCollection(innerJoinedTopt.map(lambda  feature : ee.Image.cat(feature.get('primary'),feature.get('secondary'))))

innerJoinedClim = innerJoin.apply(landsat_SR, I, filterTimeEqminmax)
landsat_SR = ee.ImageCollection(innerJoinedClim.map(lambda  feature : ee.Image.cat(feature.get('primary'),feature.get('secondary'))))

In [ ]:
# Map.addLayer(landsat_SR, {}, 'landsat_SR')
# Map

### 计算不同的环境压力因子对植被生长的影响

In [ ]:
def Calstress(image):
  I = image.select('I')  # I 代表12个月总和的热量指标
  tmean = image.select('tmean')
  pre = image.select('pr')
  Topt = image.select('Topt') # Topt 植物生长的最适温度，定义为某一区域一年内NDVI值达到最高时的当月平均气温

  # a 为因地而异的常数
  a = ee.Image().expression(
    '(0.6751 * I ** 3 - 77.1 * I ** 2 + 17920 * I + 482390) / 1000000',{'I':I}
    )
  
  # 局地潜在蒸散量
  Ep0 = ee.Image().expression(
    '16 * (10 * d / b) ** c',{'d':tmean,'b':I,'c':a}
    )

  # 地表净辐射量
  Rn = ee.Image().expression(
    '((a * b) ** 0.5) * (0.369 + 0.598 * (a / b) ** 0.5)',{'a':Ep0,'b':pre}
    )

  # 实际蒸散量
  EET = ee.Image().expression(
    'a * b * (a ** 2 + b ** 2 + a + b) / ((a + b) * (a ** 2 + b ** 2))',{'a':pre,'b':Rn}
    )

  # 潜在蒸散量
  PET = ee.Image().expression(
    '(a + b) / 2',{'a':EET,'b':Ep0}
    ).rename('PET')

  ## 水分胁迫因子
  # 水分胁迫影响系数反映了植物所能利用的有效水分条件对光能利用率的影响，随着环境中有效水分的增加逐渐增大
  Wstress = ee.Image().expression(
    '0.5 + 0.5 * (a / b)',{'a':EET,'b':PET}
    ).rename('Wstress')

  ## 温度胁迫因子
  # 在低温和高温时植物内在的生化作用对光合的限制而降低第一性生产力。
  Tstress1 = ee.Image().expression(
    '0.8 + 0.02 * a - 0.0005 * a ** 2',{'a':Topt}
    )
  Tstress1 = Tstress1.where(tmean.lt(-10),0).rename('Tstress1')

  # 表示环境温度从最适温度向高温或低温变化时植物光能利用率逐渐变小的趋势，这是因为低温和高温时高的呼吸消耗必将会降低光能利用率，生长在偏离最适温度的条件下，其光能利用率也一定会降低。
  Tsigma2 = ee.Image().expression(
    '1.184 / (1 + exp(0.2 * (a - 10 - b))) * 1 / (1 + exp(0.3 * (b - a - 10)))',{'a':Topt,'b':tmean}
    )
  Tsigma3 = ee.Image(1.184).divide(ee.Image(1).add(ee.Image(-2).exp())).multiply(ee.Image(1).divide(ee.Image(1).add(ee.Image(-3).exp())))
  diff = tmean.subtract(Topt)
  diff1 = diff.where(diff.lt(-13),0).where(diff.gte(-13).And(diff.lte(10)),1).where(diff.gt(10),0)
  # diff2 = diff.where(diff.gte(-13).And(diff.lte(10)),1).where(diff.gt(10),0).where(diff.lt(-13),0)
  diff2 = diff.where(diff.gte(-13).And(diff.lte(10)),0).where(diff.gt(10),0.5).where(diff.lt(-13),0.5).rename('diff2')
  Tstress2 = Tsigma2.multiply(diff1).add(Tsigma3.multiply(diff2)).rename('Tstress2')

  return image.addBands(PET).addBands(Wstress).addBands(Tstress1).addBands(Tstress2).clip(roi) # .addBands(PET)
landsat_SR = landsat_SR.map(Calstress)
# print(landsat_SR.size().getInfo())
# print(landsat_SR.first().bandNames().getInfo())

In [ ]:
def fun(image):
    img = image.multiply(10000).int32()
    return img
PET_images = landsat_SR.select('PET').map(fun)
# filenames =  [f'PET{i}.tif' for i in range(5,10)]
# geemap.download_ee_image_collection(PET_images, r'C:\Users\A1827\Desktop\CASA模型\PET', filenames=filenames, region=roi, scale=30, crs='epsg:4326', max_tile_dim=8000)

In [ ]:
# Map.addLayer(PET_images, {}, 'PET_images')
# Map

### 计算净初级生产力（NPP）

In [ ]:
# 计算净初级生产力
def CalNPP(image):
  NDVI = image.select('NDVI')
  SR = image.select('SR')
  NDVImin = image.select('NDVI_min')
  NDVImax = image.select('NDVI_max')
  SRmin = image.select('SR_min')
  SRmax = image.select('SR_max')
  
  # 计算FPAR（光合有效辐射分数）
  FPAR1 = NDVI.subtract(NDVImin).divide(NDVImax.subtract(NDVImin)).multiply(ee.Image(0.949)).add(0.001)
  FPAR2 = SR.subtract(SRmin).divide(SRmax.subtract(SRmin)).multiply(ee.Image(0.949)).add(0.001)
  FPAR = FPAR1.multiply(0.5).add(FPAR2.multiply(0.5))
  FPAR = FPAR.where(FPAR.gt(0.95),0.95).where(FPAR.lt(0.05),0.05).rename('FPAR')

  # 计算APAR（吸收的光合有效辐射）
  Sol = image.select('srad')
  APAR = FPAR.multiply(Sol).multiply(ee.Image(0.5)).rename('APAR')
  
  # 计算LUE（光能利用效率）与其他压力因子
  LUE = image.select('LUE')
  Wstress = image.select('Wstress')
  Tstress1 = image.select('Tstress1')
  Tstress2 = image.select('Tstress2')
  yupson = Tstress1.multiply(Tstress2).multiply(Wstress).multiply(LUE).rename('yupson')
  
  # 计算净初级生产力（NPP）
  NPP = APAR.multiply(yupson).rename('NPP')
  return image.addBands(NPP).addBands(yupson).addBands(APAR).toFloat()

# 生成月度NPP影像集合
landsat_SR = landsat_SR.map(CalNPP)
NPP_month = landsat_SR.select('NPP')

# 成年度NPP合成影像
def makeYearlyComposite_NPP(date):
  date = ee.Date(date)
  return NPP_month.filterDate(date, date.advance(1, 'year')).sum().set('system:index',date.format("YYYY")).set('year',date.get('year')).set('system:time_start',date.millis()).set('system:time_end',date.advance(1, 'year').millis())

NPP_Year = ee.ImageCollection.fromImages(years.map(makeYearlyComposite_NPP))
# print(NPP_Year.getInfo())

In [ ]:
# 分多个代码进行下载，相对于多线程，只需要修改 get 里面的数字
NPP_images = NPP_month.toList(12)

In [ ]:
for i in range(0, 12):
    NPP_image = ee.Image(NPP_images.get(i))
    month = NPP_image.get('month').getInfo()
    geemap.download_ee_image(NPP_image.clip(roi), rf"NPP_{month:02d}.tif", region=roi, scale=30, crs='epsg:4526', max_tile_size=15)